# A/B testing para ShoeFly.com

Nuestra zapatería en línea favorita, ShoeFly.com, está realizando una prueba A/B. Tienen dos versiones diferentes de un anuncio, que han colocado en correos electrónicos, así como en anuncios publicitarios en Facebook, Twitter y Google. Quieren saber cuál es el rendimiento de los dos anuncios en cada una de las diferentes plataformas en cada día de la semana. Ayúdelos a analizar los datos usando medidas agregadas.

In [21]:
import pandas as pd

# Importar datos
ad_clicks = pd.read_csv('./ad_clicks.csv')

##Análisis de fuentes de anuncios

1. Examine las primeras filas de `ad_clicks`.

In [22]:
ad_clicks.head()

,Unnamed: 0,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B


2. Su gerente quiere saber qué plataforma publicitaria le está brindando la mayor cantidad de vistas.
  ¿Cuántas vistas (es decir, filas de la tabla) provienen de cada `utm_source`?

In [23]:
ad_clicks.groupby('utm_source').user_id.count().reset_index()

,utm_source,user_id
0,email,255
1,facebook,504
2,google,680
3,twitter,215


3. Si la columna `ad_click_timestamp` no es nula, alguien realmente hizo clic en el anuncio que se mostró.

  Cree una nueva columna llamada `is_click`, que es True si `ad_click_timestamp` no es nulo y False en caso contrario.

In [24]:
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()

4. Queremos saber el porcentaje de personas que hicieron clic en los anuncios de cada `utm_source`.

  Comience agrupando por `utm_source` e `is_click` y contando el número de `user_id` en cada uno de esos grupos. Guarde su respuesta en la variable `clicks_by_source`.

In [25]:
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()

In [26]:
clicks_by_source

,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


5. Ahora vamos a pivotar los datos para que las columnas sean `is_click` (ya sea True o False), el índice sea `utm_source` y los valores sean `user_id`.

Guarde sus resultados en la variable `clicks_pivot`.

In [27]:
clicks_pivot = clicks_by_source.pivot(index = 'utm_source', columns = 'is_click', values='user_id')

In [28]:
clicks_pivot

is_click,False,True
utm_source,,
email,175,80
facebook,324,180
google,441,239
twitter,149,66


6. Cree una nueva columna en `clicks_pivot` llamada `percent_clicked`, que es igual al porcentaje de usuarios que hicieron clic en el anuncio de cada utm_source.

  ¿Hubo alguna diferencia en las tasas de clics para cada fuente?

In [29]:
clicks_pivot['percent_clicked'] = clicks_pivot[True]/(clicks_pivot[True] + clicks_pivot[False])

In [30]:
clicks_pivot

is_click,False,True,percent_clicked
utm_source,,,
email,175,80,0.313725
facebook,324,180,0.357143
google,441,239,0.351471
twitter,149,66,0.306977


##Análisis de un test A/B

7. La columna experimental_group nos dice si al usuario se le mostró el Anuncio A o el Anuncio B.

  ¿Se mostró aproximadamente el mismo número de personas en ambos anuncios?

In [31]:
ad_clicks.groupby('experimental_group').user_id.count().reset_index()

,experimental_group,user_id
0,A,827
1,B,827


8. Usando la columna is_click que definimos anteriormente, verifique si un mayor porcentaje de usuarios hizo clic en el Anuncio A o el Anuncio B.

In [32]:
ad_clicks.groupby(['experimental_group','is_click']).user_id.count()\
  .reset_index().pivot(index='experimental_group',columns='is_click',values='user_id').reset_index()

is_click,experimental_group,False,True
0,A,517,310
1,B,572,255


9. El gerente de producto del test A/B cree que los clics pueden haber cambiado según el día de la semana.

  Comience creando dos DataFrames: `a_clicks` y `b_clicks`, que contienen solo los resultados para el grupo A y el grupo B, respectivamente.

In [33]:
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']

10. Para cada grupo (`a_clicks` y `b_clicks`), calcule el porcentaje de usuarios que hicieron clic en el anuncio por día.

In [34]:
a_clicks_pivot = a_clicks.groupby(['is_click','day']).user_id.count().reset_index()\
  .pivot(index='day',columns='is_click',values='user_id').reset_index()

a_clicks_pivot['percent_clicked'] = a_clicks_pivot[True] / (a_clicks_pivot[True] + a_clicks_pivot[False])

In [35]:
a_clicks_pivot

is_click,day,False,True,percent_clicked
0,1 - Monday,70,43,0.380531
1,2 - Tuesday,76,43,0.361345
2,3 - Wednesday,86,38,0.306452
3,4 - Thursday,69,47,0.405172
4,5 - Friday,77,51,0.398438
5,6 - Saturday,73,45,0.381356
6,7 - Sunday,66,43,0.394495


In [36]:
b_clicks_pivot = b_clicks.groupby(['is_click','day']).user_id.count().reset_index()\
  .pivot(index='day',columns='is_click',values='user_id').reset_index()

b_clicks_pivot['percent_clicked'] = b_clicks_pivot[True] / (b_clicks_pivot[True] + b_clicks_pivot[False])

In [37]:
b_clicks_pivot

is_click,day,False,True,percent_clicked
0,1 - Monday,81,32,0.283186
1,2 - Tuesday,74,45,0.378151
2,3 - Wednesday,89,35,0.282258
3,4 - Thursday,87,29,0.250000
4,5 - Friday,90,38,0.296875
5,6 - Saturday,76,42,0.355932
6,7 - Sunday,75,34,0.311927


11. Compara los resultados de A y B. ¿Qué pasó en el transcurso de la semana?
  ¿Recomienda que su empresa utilice el Anuncio A o el Anuncio B?

  **Respuesta:** se recomienda el uso del anuncio a dado su mayor porcentaje de clicks activos.